In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os

In [6]:
folder_path = '/Users/matteoferrazzi/Documents/GitHub/ML-project-2/Data/monthly_data'

files = os.listdir(folder_path)

In [7]:
datasets = []

for file in files:
    if file == '.DS_Store':
        continue
    df = pd.read_csv(folder_path + '/' + file, encoding='utf-8')
    df['name'] = file
    datasets.append(df)

datasets.sort(key=lambda x: x['name'][0])

In [8]:
macro = pd.read_csv('Data/macro_data_amit_goyal.csv', encoding='utf-8')
macro = macro[macro['yyyymm']>199000]

In [9]:
def standardize(df):
    return (df - df.mean()) / df.std()

In [10]:
def fill_missing(df):
    return df.fillna(0)

In [11]:
data = []
ret = []

for i,df in enumerate(datasets):
    
    df['mcap'] = df['SHROUT'] * df['prc']
    df.drop(['permno', 'DATE', 'Unnamed: 0', 'mve0', 'prc', 'SHROUT', 'sic2', 'name'], axis=1, inplace=True)
    df.dropna(thresh=60, axis=0, inplace=True)
    df = df[df['RET'] > -1]
    df = df.sort_values(by=['mcap'], ascending=False)
    df.drop(['mcap'], axis=1, inplace=True)
    df = df.head(100)
    ret.append(df['RET']-macro['Rfree'].values[i])
    df = df.drop(['RET'], axis=1)
    df = standardize(df)
    df = fill_missing(df)
    data.append(df)

In [12]:
def solve_f(ret, data, gamma, idx):
    # risolve per la singola f poi dobbiamo metterle in una lista
    return np.linalg.solve(gamma.T@data[idx-1].values.T@data[idx-1].values@gamma, gamma.T@data[idx-1].values.T@ret[idx].values)

In [23]:
def solve_gamma(ret, data, f):
    # f viene passato come lista
    A = [np.kron(data[i].values.T@data[i].values, f[i].reshape(-1,1)@f[i].reshape(1,-1)) for i in range(len(data)-1)]
    B = [np.kron(data[i].values,f[i].reshape((1,-1))).T@ret[i+1] for i in range(len(data)-1)]
    print(A.shape, B.shape)
    vec_gamma = np.linalg.solve(A, B)
    return vec_gamma.reshape((94, len(f[0])))

In [14]:
f_list = []
for i in range(len(data)-1):
    f_list.append(solve_f(ret, data, np.random.rand(94,5), i+1))

In [24]:
gamma = solve_gamma(ret, data, f_list)

() ()


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [20]:
# test tensor product
A = np.random.rand(2,3)
B = np.random.rand(3)
B.shape
np.kron(A,B).shape

(2, 9)

In [ ]:
import torch